## Granger 인과성

In [2]:
# 검정 방향:
# 반도체 ETF → 삼성전자
# 삼성전자 → 반도체 ETF

In [ ]:
# 종목	설명
# 005930	삼성전자 (KOSPI)
# 091160	KODEX 반도체 ETF (국내 상장, 반도체 업황 지표 대용 가능)

In [1]:
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import grangercausalitytests
from pykrx import stock
import datetime

# 1. 날짜 범위 설정
start = "2023-01-01"
end = "2025-06-25"
start_fmt = datetime.datetime.strptime(start, "%Y-%m-%d").strftime("%Y%m%d")
end_fmt = datetime.datetime.strptime(end, "%Y-%m-%d").strftime("%Y%m%d")

# 2. 삼성전자 주가 데이터 가져오기
samsung = stock.get_market_ohlcv_by_date(start_fmt, end_fmt, "005930")
samsung['log_return'] = np.log(samsung['종가'] / samsung['종가'].shift(1))

# 3. KODEX 반도체 ETF (국내 대표 반도체 업황 ETF) 데이터 가져오기
semiconductor = stock.get_market_ohlcv_by_date(start_fmt, end_fmt, "091160")  # KODEX 반도체
semiconductor['log_return'] = np.log(semiconductor['종가'] / semiconductor['종가'].shift(1))

# 4. 병합 및 정리
data = pd.concat([samsung['log_return'], semiconductor['log_return']], axis=1)
data.columns = ['samsung_ret', 'semi_ret']
data.dropna(inplace=True)

# 5. Granger Causality Test
print("\n📈 Granger Causality Test: Does Semiconductor lead Samsung?")
grangercausalitytests(data[['samsung_ret', 'semi_ret']], maxlag=5, verbose=True)

print("\n📈 Granger Causality Test: Does Samsung lead Semiconductor?")
grangercausalitytests(data[['semi_ret', 'samsung_ret']], maxlag=5, verbose=True)


C:\Users\JYB\AppData\Local\Programs\Python\Python39\lib\site-packages\pykrx\__init__.py:4: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources



📈 Granger Causality Test: Does Semiconductor lead Samsung?

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.2275  , p=0.6336  , df_denom=599, df_num=1
ssr based chi2 test:   chi2=0.2286  , p=0.6325  , df=1
likelihood ratio test: chi2=0.2286  , p=0.6326  , df=1
parameter F test:         F=0.2275  , p=0.6336  , df_denom=599, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=1.1152  , p=0.3285  , df_denom=596, df_num=2
ssr based chi2 test:   chi2=2.2490  , p=0.3248  , df=2
likelihood ratio test: chi2=2.2448  , p=0.3255  , df=2
parameter F test:         F=1.1152  , p=0.3285  , df_denom=596, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=0.7605  , p=0.5165  , df_denom=593, df_num=3
ssr based chi2 test:   chi2=2.3085  , p=0.5109  , df=3
likelihood ratio test: chi2=2.3041  , p=0.5117  , df=3
parameter F test:         F=0.7605  , p=0.5165  , df_denom=593, df_num=3

Granger Causality
number of lags 

C:\Users\JYB\AppData\Local\Programs\Python\Python39\lib\site-packages\statsmodels\tsa\stattools.py:1545: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
C:\Users\JYB\AppData\Local\Programs\Python\Python39\lib\site-packages\statsmodels\tsa\stattools.py:1545: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(


{1: ({'ssr_ftest': (0.10666433863086919, 0.7440882563944562, 599.0, 1),
   'ssr_chi2test': (0.10719855067743449, 0.743356368963896, 1),
   'lrtest': (0.1071890073517352, 0.7433673906537899, 1),
   'params_ftest': (0.10666433863105289, 0.744088256394225, 599.0, 1.0)},
   array([[0., 1., 0.]])]),
 2: ({'ssr_ftest': (1.0898354199527327, 0.3369410091347403, 596.0, 2),
   'ssr_chi2test': (2.1979566690993035, 0.33321134036835937, 2),
   'lrtest': (2.1939472953990844, 0.3338799947581036, 2),
   'params_ftest': (1.0898354199526665, 0.3369410091347594, 596.0, 2.0)},
   array([[0., 0., 1., 0., 0.],
          [0., 0., 0., 1., 0.]])]),
 3: ({'ssr_ftest': (0.7684401337663683, 0.511986824828556, 593.0, 3),
   'ssr_chi2test': (2.3325332896786897, 0.506317058634483, 3),
   'lrtest': (2.3280110798277747, 0.5071759770685238, 3),
   'params_ftest': (0.768440133766387, 0.511986824828556, 593.0, 3.0)},
   array([[0., 0., 0., 1., 0., 0., 0.],
          [0., 0., 0., 0., 1., 0., 0.],
          [0., 0., 0., 0.

In [ ]:
# 결과 해석 팁
# 각 시차(lag)에 대해 F-test의 P-value가 출력됩니다.

# P < 0.05이면 통계적으로 유의미 → X가 Y를 Granger-cause 한다고 해석

In [ ]:
# Lags	반도체 → 삼성전자	삼성전자 → 반도체
# 1 	p = 0.6336	p = 0.7441
# 2 	p = 0.3285	p = 0.3369
# 3	   p = 0.5165	p = 0.5120
# 4	   p = 0.5735	p = 0.2087
# 5	   p = 0.6679	p = 0.2850

# 기준: p < 0.05 → 통계적으로 유의한 Granger 인과성 존재

In [ ]:
# 해석
# 1. 반도체 → 삼성전자 (예측 가능성 검정)
# 모든 시차(lag)에서 p > 0.05
# 즉, 반도체 ETF의 수익률이 삼성전자 수익률을 통계적으로 유의미하게 예측하지 않음
# → "Granger 인과성 없음"으로 판단
# 2. 삼성전자 → 반도체
# 마찬가지로 모든 시차에서 p > 0.05
# 삼성전자 주가 수익률 역시 반도체 ETF 수익률을 Granger-cause 하지 않음

## 정상성 자료로 변환후 검정

### 정상성 검정

In [ ]:
# Granger Causality Test를 정확하게 수행하려면 두 시계열이 정상성(stationarity)을 만족해야 하므로, 
# ADF (Augmented Dickey-Fuller) 테스트를 통해 정상성을 검정하고, 
# 필요한 경우 차분(differencing)을 수행해야 합니다.

In [3]:
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import grangercausalitytests, adfuller
from pykrx import stock
import datetime

# 1. 날짜 범위 설정
start = "2023-01-01"
end = "2025-06-25"
start_fmt = datetime.datetime.strptime(start, "%Y-%m-%d").strftime("%Y%m%d")
end_fmt = datetime.datetime.strptime(end, "%Y-%m-%d").strftime("%Y%m%d")

# 2. 삼성전자 및 반도체 ETF 데이터 가져오기
samsung = stock.get_market_ohlcv_by_date(start_fmt, end_fmt, "005930")
semi = stock.get_market_ohlcv_by_date(start_fmt, end_fmt, "091160")  # KODEX 반도체

# 3. 로그 수익률 계산
samsung['log_return'] = np.log(samsung['종가'] / samsung['종가'].shift(1))
semi['log_return'] = np.log(semi['종가'] / semi['종가'].shift(1))

# 4. 병합 및 결측 제거
data = pd.concat([samsung['log_return'], semi['log_return']], axis=1)
data.columns = ['samsung_ret', 'semi_ret']
data.dropna(inplace=True)

# 5. ADF 테스트 함수
def adf_test(series, name):
    result = adfuller(series, autolag='AIC')
    print(f"\n📉 ADF Test for {name}")
    print(f"ADF Statistic: {result[0]}")
    print(f"p-value: {result[1]}")
    print("→ " + ("정상성 있음 (stationary)" if result[1] < 0.05 else "정상성 없음 (non-stationary)"))

# 6. ADF 테스트 수행
adf_test(data['samsung_ret'], "Samsung")
adf_test(data['semi_ret'], "Semiconductor")

# 7. (Optional) 차분 - 이미 로그 수익률이면 대부분 정상성이므로 생략 가능
# 만약 비정상성이 확인된다면 아래 코드 주석 제거하여 1차 차분 수행
# data['samsung_ret'] = data['samsung_ret'].diff()
# data['semi_ret'] = data['semi_ret'].diff()
# data.dropna(inplace=True)

# 8. Granger Causality Test
print("\n📈 Granger Causality Test: Does Semiconductor lead Samsung?")
grangercausalitytests(data[['samsung_ret', 'semi_ret']], maxlag=5, verbose=True)

print("\n📈 Granger Causality Test: Does Samsung lead Semiconductor?")
grangercausalitytests(data[['semi_ret', 'samsung_ret']], maxlag=5, verbose=True)



📉 ADF Test for Samsung
ADF Statistic: -18.30147673196317
p-value: 2.2861005537889714e-30
→ 정상성 있음 (stationary)

📉 ADF Test for Semiconductor
ADF Statistic: -23.513452548233253
p-value: 0.0
→ 정상성 있음 (stationary)

📈 Granger Causality Test: Does Semiconductor lead Samsung?

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.2275  , p=0.6336  , df_denom=599, df_num=1
ssr based chi2 test:   chi2=0.2286  , p=0.6325  , df=1
likelihood ratio test: chi2=0.2286  , p=0.6326  , df=1
parameter F test:         F=0.2275  , p=0.6336  , df_denom=599, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=1.1152  , p=0.3285  , df_denom=596, df_num=2
ssr based chi2 test:   chi2=2.2490  , p=0.3248  , df=2
likelihood ratio test: chi2=2.2448  , p=0.3255  , df=2
parameter F test:         F=1.1152  , p=0.3285  , df_denom=596, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=0.7605  , p=0.5165  , df_denom=593, df_num=3
ssr b

C:\Users\JYB\AppData\Local\Programs\Python\Python39\lib\site-packages\statsmodels\tsa\stattools.py:1545: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
C:\Users\JYB\AppData\Local\Programs\Python\Python39\lib\site-packages\statsmodels\tsa\stattools.py:1545: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(


{1: ({'ssr_ftest': (0.10666433863086919, 0.7440882563944562, 599.0, 1),
   'ssr_chi2test': (0.10719855067743449, 0.743356368963896, 1),
   'lrtest': (0.1071890073517352, 0.7433673906537899, 1),
   'params_ftest': (0.10666433863105289, 0.744088256394225, 599.0, 1.0)},
   array([[0., 1., 0.]])]),
 2: ({'ssr_ftest': (1.0898354199527327, 0.3369410091347403, 596.0, 2),
   'ssr_chi2test': (2.1979566690993035, 0.33321134036835937, 2),
   'lrtest': (2.1939472953990844, 0.3338799947581036, 2),
   'params_ftest': (1.0898354199526665, 0.3369410091347594, 596.0, 2.0)},
   array([[0., 0., 1., 0., 0.],
          [0., 0., 0., 1., 0.]])]),
 3: ({'ssr_ftest': (0.7684401337663683, 0.511986824828556, 593.0, 3),
   'ssr_chi2test': (2.3325332896786897, 0.506317058634483, 3),
   'lrtest': (2.3280110798277747, 0.5071759770685238, 3),
   'params_ftest': (0.768440133766387, 0.511986824828556, 593.0, 3.0)},
   array([[0., 0., 0., 1., 0., 0., 0.],
          [0., 0., 0., 0., 1., 0., 0.],
          [0., 0., 0., 0.

In [ ]:
# 결과 해석 방법
# ADF 테스트의 p-value < 0.05 → 정상성 있음 (차분 불필요)

# ADF 테스트의 p-value ≥ 0.05 → 정상성 없음 → 1차 차분 후 다시 검정 필요

# 수익률 데이터는 일반적으로 이미 정상성을 가지므로 차분 없이 Granger 테스트 가능

### 차분 적용 + Granger 인과성 테스트)

In [4]:
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import grangercausalitytests, adfuller
from pykrx import stock
import datetime

# 1. 날짜 범위 설정
start = "2023-01-01"
end = "2025-06-25"
start_fmt = datetime.datetime.strptime(start, "%Y-%m-%d").strftime("%Y%m%d")
end_fmt = datetime.datetime.strptime(end, "%Y-%m-%d").strftime("%Y%m%d")

# 2. 삼성전자 및 반도체 ETF 데이터 가져오기
samsung = stock.get_market_ohlcv_by_date(start_fmt, end_fmt, "005930")
semi = stock.get_market_ohlcv_by_date(start_fmt, end_fmt, "091160")  # KODEX 반도체

# 3. 로그 수익률 계산
samsung['log_return'] = np.log(samsung['종가'] / samsung['종가'].shift(1))
semi['log_return'] = np.log(semi['종가'] / semi['종가'].shift(1))

# 4. 병합 및 결측 제거
data = pd.concat([samsung['log_return'], semi['log_return']], axis=1)
data.columns = ['samsung_ret', 'semi_ret']
data.dropna(inplace=True)

# 5. 1차 차분 적용 (수익률에 대해 한 번 더 차분)
data['samsung_diff'] = data['samsung_ret'].diff()
data['semi_diff'] = data['semi_ret'].diff()
data.dropna(inplace=True)

# 6. ADF 정상성 검정 (차분 후)
def adf_test(series, name):
    result = adfuller(series, autolag='AIC')
    print(f"\n📉 ADF Test for {name} (차분 적용됨)")
    print(f"ADF Statistic: {result[0]}")
    print(f"p-value: {result[1]}")
    print("→ " + ("정상성 있음" if result[1] < 0.05 else "정상성 없음"))

adf_test(data['samsung_diff'], "삼성전자 수익률 차분")
adf_test(data['semi_diff'], "반도체 ETF 수익률 차분")

# 7. Granger 인과성 검정 (차분된 시계열 사용)
print("\n📈 Granger Causality Test: 반도체 ETF → 삼성전자")
grangercausalitytests(data[['samsung_diff', 'semi_diff']], maxlag=5, verbose=True)

print("\n📈 Granger Causality Test: 삼성전자 → 반도체 ETF")
grangercausalitytests(data[['semi_diff', 'samsung_diff']], maxlag=5, verbose=True)



📉 ADF Test for 삼성전자 수익률 차분 (차분 적용됨)
ADF Statistic: -10.821886378631223
p-value: 1.7968925520905104e-19
→ 정상성 있음

📉 ADF Test for 반도체 ETF 수익률 차분 (차분 적용됨)
ADF Statistic: -10.10835961251772
p-value: 1.0157102556149504e-17
→ 정상성 있음

📈 Granger Causality Test: 반도체 ETF → 삼성전자

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.2559  , p=0.6132  , df_denom=598, df_num=1
ssr based chi2 test:   chi2=0.2572  , p=0.6121  , df=1
likelihood ratio test: chi2=0.2571  , p=0.6121  , df=1
parameter F test:         F=0.2559  , p=0.6132  , df_denom=598, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.6310  , p=0.5324  , df_denom=595, df_num=2
ssr based chi2 test:   chi2=1.2725  , p=0.5293  , df=2
likelihood ratio test: chi2=1.2712  , p=0.5296  , df=2
parameter F test:         F=0.6310  , p=0.5324  , df_denom=595, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=0.6526  , p=0.5816  , df_denom=592, df_num=3
ssr bas

C:\Users\JYB\AppData\Local\Programs\Python\Python39\lib\site-packages\statsmodels\tsa\stattools.py:1545: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
C:\Users\JYB\AppData\Local\Programs\Python\Python39\lib\site-packages\statsmodels\tsa\stattools.py:1545: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(


{1: ({'ssr_ftest': (0.9072668871047509, 0.34122546806444587, 598.0, 1),
   'ssr_chi2test': (0.9118183932273499, 0.33963240543023876, 1),
   'lrtest': (0.9111274008746477, 0.3398154629682342, 1),
   'params_ftest': (0.9072668871049665, 0.34122546806437504, 598.0, 1.0)},
   array([[0., 1., 0.]])]),
 2: ({'ssr_ftest': (1.4462409536646792, 0.23628017658248054, 595.0, 2),
   'ssr_chi2test': (2.916788477979185, 0.2326094902521957, 2),
   'lrtest': (2.9097216588897936, 0.23343284861976246, 2),
   'params_ftest': (1.4462409536647118, 0.23628017658248054, 595.0, 2.0)},
   array([[0., 0., 1., 0., 0.],
          [0., 0., 0., 1., 0.]])]),
 3: ({'ssr_ftest': (1.497060503208653, 0.21426060323021268, 592.0, 3),
   'ssr_chi2test': (4.544286696395185, 0.20837386901055868, 3),
   'lrtest': (4.52713587058679, 0.20988256345547854, 3),
   'params_ftest': (1.497060503208709, 0.2142606032301988, 592.0, 3.0)},
   array([[0., 0., 0., 1., 0., 0., 0.],
          [0., 0., 0., 0., 1., 0., 0.],
          [0., 0., 0